In [1]:
#import libraries
import torch
import torch.nn as nn
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as dsets
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import collections


#Load train dataset of MNIST
Train_dataset = dsets.MNIST(root ='./data',
                            train = True,
                            transform = transforms.ToTensor(),
                            download = True)

#Load test dataset of MNIST
Test_dataset = dsets.MNIST(root = './data',
                           train = False,
                           transform = transforms.ToTensor())


n_train =len(Train_dataset)
n_test= len(Test_dataset)
print("Train Dataset: ",n_train)
print("Test Dataset:  ",n_test)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Train Dataset:  60000
Test Dataset:   10000


In [2]:
#defining epochs
batch_size = 100
n_iterable = 3000
num_epochs = n_iterable / (len(Train_dataset)/batch_size)
num_epochs = int(num_epochs)
Train_Loader = torch.utils.data.DataLoader(dataset=Train_dataset,
                                      batch_size = batch_size,
 
                                      shuffle = True)
Test_Loader = torch.utils.data.DataLoader(dataset=Test_dataset,
                                          batch_size = batch_size,
                                          shuffle = False)
# check iterability
itb_train = isinstance(Train_Loader, collections.Iterable)
itb_test  = isinstance(Test_Loader, collections.Iterable)
print(itb_train)
print(itb_test)

True
True


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  


In [3]:
#Building Model
class LogisticRegressionModel(nn.Module):
  def __init__(self, input_dim,output_dim):
    super(LogisticRegressionModel, self).__init__()
    self.linear = nn.Linear(input_dim, output_dim)
    

  def forward(self,x):
    out = self.linear(x)
    return out


In [4]:
input_dim = 28*28   #from model
output_dim = 10
model = LogisticRegressionModel(input_dim,output_dim)

#Create Cross Entry Loss Class
criterion = nn.CrossEntropyLoss()

In [6]:
"""
 Instantiate Optimizer Class
 θ = θ − η⋅∇θ
 θ = parameters
 η = learning rate
 ∇θ = parameters' gradients
"""
# with regularization
learning_rate = 0.001
weight = 0.01

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=weight)
# with ADAM
#optimizer = torch.optim.ADAM(model.parameters(), lr=learning_rate, weight_decay = weight)

#with NESTEROV
#optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate,weight_decay = weight,momentum = 0.9,nesterov=True)

# Train Model
iteration = 0
for epoch in range(num_epochs):
  for i, (images,labels) in enumerate(Train_Loader):
    images = images.view(-1,28*28).requires_grad_()
    labels = labels

        # Clear gradients w.r.t. parameters
    optimizer.zero_grad()
        # Forward pass to get output/logits
    outputs = model(images)

        # Calculate Loss: softmax --> cross entropy loss
    loss = criterion(outputs, labels)

        # Getting gradients w.r.t. parameters
    loss.backward()

        # Updating parameters
    optimizer.step()

    iteration += 1

    if iteration % 500 == 0:
      correct = 0
      total = 0
          # Iterate test dataset
      for images,labels in Test_Loader:
        images = images.view(-1, 28*28).requires_grad_()

                # Forward pass only to get logits/output
        outputs = model(images)

                # Get predictions from the maximum value
        _, predicted = torch.max(outputs.data, 1)

                # Total number of labels
        total += labels.size(0)

                # Total correct predictions
        correct += (predicted == labels).sum()

      accuracy = 100 * correct / total

        # Loss
      print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iteration, loss.item(), accuracy))


Iteration: 500. Loss: 0.9730315208435059. Accuracy: 83.48999786376953
Iteration: 1000. Loss: 0.8275304436683655. Accuracy: 83.97000122070312
Iteration: 1500. Loss: 0.8892596960067749. Accuracy: 84.41000366210938
Iteration: 2000. Loss: 0.8016198873519897. Accuracy: 84.8499984741211
Iteration: 2500. Loss: 0.9457733035087585. Accuracy: 85.29000091552734
Iteration: 3000. Loss: 0.715333104133606. Accuracy: 85.63999938964844


In [7]:
# ACCURACY
correct = 0
total = 0
iter_test = 0
for images, labels in Test_Loader:
    iter_test += 1
    images = images.view(-1, 28*28).requires_grad_()
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)

    # Total number of labels
    total += labels.size(0)

    # Total correct predictions
    correct += (predicted == labels).sum()

accuracy = 100 * (correct.item() / total)

print(accuracy)

85.64
